In [ ]:
# Extract results

In [ ]:
### Imports 
import os
import math
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.patches as patches
from matplotlib.collections import LineCollection
import time
import json

import pandas as pd

sys.path.append('../../')
from script.conversion.bison.coordinates import rd_to_utm
from mnms.graph.layers import PublicTransportLayer, MultiLayerGraph, OriginDestinationLayer, SharedVehicleLayer
from mnms.generation.roads import generate_pt_line_road, generate_one_zone
from mnms.generation.layers import generate_layer_from_roads
from mnms.vehicles.veh_type import Tram, Metro, Bus, Bike
from mnms.generation.zones import generate_one_zone
from mnms.mobility_service.public_transport import PublicTransportMobilityService
from mnms.mobility_service.vehicle_sharing import VehicleSharingMobilityService
from mnms.time import TimeTable, Dt, Time
from mnms.io.graph import load_graph, save_graph, load_odlayer, save_transit_links
from mnms.tools.render import draw_roads, draw_line, draw_odlayer, draw_path, draw_veh_activity
#from mnms.tools.geometry import points_in_polygon, get_bounding_box

In [ ]:
### Parameters

# Files and directories
f = open('params.json')
params = json.load(f)

current_dir = os.getcwd()
indir = current_dir + '/inputs/'
outdir = current_dir + '/outputs/'

#coord_csv_filepath = indir + 'KV1_GVB_2609_2/Csv/POINT.csv' # file with coordinates of the network
#amsterdam_json_filepath = indir + 'new_network.json' # mlgraph with the road network only
#amsterdam_json_filepath_pt_transit = indir + "network_pt_transit.json"
#transit_path = indir + "transit.json"

# Transit connection (m)
DIST_MAX = 500
DIST_CONNECTION_OD = 200
DIST_CONNECTION_PT = 100
DIST_CONNECTION_MIX = 100

In [ ]:
### Load network

mmgraph_pt = load_graph(indir + params["fn_network"])
df_stations = pd.read_csv(indir+params['fn_emoped_st_init'])

#df_emoped1 = pd.read_csv(indir + 'init_pos_emoped.csv')
#df_emoped2 = pd.read_csv(indir + 'init_pos_emoped.csv')

### Load demand

df_dmd = pd.read_csv(indir + params['fn_demand'], sep=";")

### Load odlayer
odlayer = load_odlayer(indir + params["fn_odlayer"])
x_od = []
y_od = []
for i in odlayer.origins.items():
    x = i[1][0]
    y = i[1][1]
    x_od.append(x)
    y_od.append(y)

In [ ]:
## Load results

df_emoped1 = pd.read_csv(outdir+'emoped1_veh.csv', sep=";")
df_path = pd.read_csv(outdir+'path.csv', sep=";")

In [ ]:
# Look vehicle trajectory
veh_id = 47
df = df_emoped1[df_emoped1["ID"] == int(veh_id)]

In [ ]:
# Look O/D for users wihtout paths
list_nomatch = []
list_origins = []
list_destinations = []
for i, row in df_path[:].iterrows():
    if pd.isna(row['PATH']):
        list_nomatch.append(row['ID'])
        user = df_dmd[df_dmd['ID']==row['ID']]
        list_origins.append([float(o) for o in user['ORIGIN'].iloc[0].split(' ')])
        list_destinations.append([float(o) for o in user['DESTINATION'].iloc[0].split(' ')])

In [ ]:
df_dmd

In [ ]:
id_user = '1063608-1'
o_user =  [float(o) for o in df_dmd[df_dmd['ID']==id_user]['ORIGIN'].iloc[0].split(' ')]
d_user =  [float(o) for o in df_dmd[df_dmd['ID']==id_user]['DESTINATION'].iloc[0].split(' ')]

dist_o = [(o_user[0]-x)**2 + (o_user[1]-y)**2 for (x,y) in zip(x_od,y_od)]
dist_d = [(d_user[0]-x)**2 + (d_user[1]-y)**2 for (x,y) in zip(x_od,y_od)]
print(np.argmin(dist_o), np.argmin(dist_d))

In [ ]:
polygon = np.asarray([
    [628000, 5.81e6],
    [632500, 5.808e6],
    [634000, 5.8055e6],
    [634000, 5.803e6],
    [633000, 5.801e6],
    [630500, 5.7995e6],
    [629000, 5.8e6],
    [625500, 5.8e6],
    [625500, 5.803e6],
    [625500, 5.806e6]
])

In [ ]:
x_st = []
y_st = []
for n in df_stations['closest_node']:
    x_st.append(mmgraph_pt.roads.nodes[n].position[0])
    y_st.append(mmgraph_pt.roads.nodes[n].position[1])

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
draw_roads(ax, mmgraph_pt.roads, color='grey', linkwidth=0.1, nodesize=0, draw_stops=False, node_label=False)
for i, row in df.iterrows():
    pos = row['POSITION'].split(' ')
    plt.text(float(pos[0]), float(pos[1]), row.TIME)

for o,d in zip(list_origins, list_destinations):
    plt.plot([o[0], d[0]], [o[1], d[1]], ':')
    plt.plot(d[0], d[1], 'dg')
    plt.plot(o[0], o[1], 'or')

plt.plot([o_user[0], d_user[0]], [o_user[1], d_user[1]], 'o-')

plt.plot(x_od, y_od, '*b')

plt.plot(x_st, y_st, 'dk', alpha=0.4)

plt.plot(polygon[:,0], polygon[:,1], '+--k')